# Problem Statement
### Upgrade propensity model

In this task, the goal is to predict the overall rating of customers using the review and summary information.

### Task list
Data exploration, data cleaning and feature engineering. Remember to comment each step properly.
Explanatory analysis. Include key insights in the presentation
Modelling/Training
Prepare a presentation and present the analysis. Use visualisations and tell compelling stories. Tell us what is the message(s) for the business? What actionable insights you can provide to the business?

### Required Output
Fully commented code
Modelling dataset (json file)
Visualisation tool files (if anything)
Slides of presentations, including but not limited to:
Problem statement, Assumptions, Hypotheses
Exploratory analysis
Modelling approach
Results
Model performance measures
Business audience performance measures
Insights
Recommendations

# Data Exploration and Preparation

## Loading the data

- This section includes loading of the data, understanding basic description of the data; followed by cleanup & imputations if required.

In [1]:
# Importing the required packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from collections import Counter

import re
import string

import nltk
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score,precision_score,recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
import xgboost as xgb
# import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading at the data and looking at the shape

review_data = pd.read_json('data/Musical_Instruments_5.json', lines=True)
print('Shape of data is:', review_data.shape)

Shape of data is: (10261, 9)


In [3]:
# review_data.head()

## Variable Identification

In [4]:
# # Looking at individual features
# review_data.dtypes

In [5]:
# # Checking the count of unique target values

# sns.countplot(review_data['overall']);
# # c = Counter(review_data['overall'])
# # [(i, c[i] / len(review_data['overall']) * 100.0) for i, count in c.most_common()]

In [6]:
# # Checking for unique values

# review_data.loc[:,['reviewerID', 'asin', 'reviewerName', 'reviewText', 'overall', 'summary']].nunique()

In [7]:
# # Checking nan values across dataset

# review_data.isna().sum()

In [8]:
# # Removing uneccessary columns for now

# review_data.drop(labels=['reviewerID', 'asin', 'reviewerName', 'helpful', 'unixReviewTime', 'reviewTime'], axis=1, inplace=True)

In [9]:
# Combining summary and  reviewText

review_data['text'] = review_data['summary'] + ' ' + review_data['reviewText']
review_data.drop(labels=['summary', 'reviewText'], axis=1, inplace=True)
review_data.head()

,reviewerID,asin,reviewerName,helpful,overall,unixReviewTime,reviewTime,text
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]",5,1393545600,"02 28, 2014","good Not much to write about here, but it does..."
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",5,1363392000,"03 16, 2013",Jake The product does exactly as it should and...
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",5,1377648000,"08 28, 2013",It Does The Job Well The primary job of this d...
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",5,1392336000,"02 14, 2014",GOOD WINDSCREEN FOR THE MONEY Nice windscreen ...
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",5,1392940800,"02 21, 2014",No more pops when I record my vocals. This pop...


In [10]:
review_data.shape

(10261, 8)

#### Initial Observations:

- There are a total of 10261 records and 9 features. However, for this problem we shall be taking into account the summary, reviewtext and overall.
- There is noticable imbalance 5 and 4 ratings at 87 percent and the remaining 13 percent are covered with 1,2,3 ratings. 

In [11]:
review_data['text'][1]

"Jake The product does exactly as it should and is quite affordable.I did not realized it was double screened until it arrived, so it was even better than I had expected.As an added bonus, one of the screens carries a small hint of the smell of an old grape candy I used to buy, so for reminiscent's sake, I cannot stop putting the pop filter next to my nose and smelling it after recording. :DIf you needed a pop filter, this will work just as well as the expensive ones, and it may even come with a pleasing aroma like mine did!Buy this product! :]"

### Data Preprocessing

In [12]:
# function to remove punctuations
# string.punctuation
def remove_punctuation(text):
#     no_punct = "".join([c for c in text if c not in string.punctuation])
#     return no_punct
    string1 = text.lower()
    translation_table = dict.fromkeys(map(ord, string.punctuation),' ')
    string2 = string1.translate(translation_table)
    return string2

review_data['text'] = review_data['text'].apply(lambda x: remove_punctuation(x))
review_data['text'][1]

'jake the product does exactly as it should and is quite affordable i did not realized it was double screened until it arrived  so it was even better than i had expected as an added bonus  one of the screens carries a small hint of the smell of an old grape candy i used to buy  so for reminiscent s sake  i cannot stop putting the pop filter next to my nose and smelling it after recording   dif you needed a pop filter  this will work just as well as the expensive ones  and it may even come with a pleasing aroma like mine did buy this product    '

In [13]:
# function to remove stop words
# stopwords.words('english')

def remove_stopwords(text):
#     return [w for w in text if w not in stopwords.words('english')]
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    string2 = pattern.sub(' ', text)
    return string2

review_data['text'] = review_data['text'].apply(lambda x: remove_stopwords(x))
review_data['text'][1]

'jake  product  exactly      quite affordable    realized   double screened   arrived     even better    expected   added bonus  one   screens carries  small hint   smell   old grape candy  used  buy    reminiscent  sake   cannot stop putting  pop filter next   nose  smelling   recording   dif  needed  pop filter    work   well   expensive ones    may even come   pleasing aroma like mine  buy  product    '

In [14]:
# function to tokenize
def tokenize_words(text):
    return nltk.word_tokenize(text)

review_data['text'] = review_data['text'].apply(lambda x: tokenize_words(x.lower()))
review_data['text'][1]

['jake',
 'product',
 'exactly',
 'quite',
 'affordable',
 'realized',
 'double',
 'screened',
 'arrived',
 'even',
 'better',
 'expected',
 'added',
 'bonus',
 'one',
 'screens',
 'carries',
 'small',
 'hint',
 'smell',
 'old',
 'grape',
 'candy',
 'used',
 'buy',
 'reminiscent',
 'sake',
 'can',
 'not',
 'stop',
 'putting',
 'pop',
 'filter',
 'next',
 'nose',
 'smelling',
 'recording',
 'dif',
 'needed',
 'pop',
 'filter',
 'work',
 'well',
 'expensive',
 'ones',
 'may',
 'even',
 'come',
 'pleasing',
 'aroma',
 'like',
 'mine',
 'buy',
 'product']

In [15]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

#Lemmatization
def lemmatize_words(data_str):
# expects a string
    list_pos = 0
    cleaned_str = ''
    final_text=[]
    lmtzr = WordNetLemmatizer()
    stop = set(stopwords.words('english'))
    punc = list(string.punctuation)
    stop.update(punc)
    for i in data_str:
        if i.strip().lower() not in stop:
            pos = pos_tag([i.strip()])
            if get_simple_pos(pos[0][1]) is None:
                pass
            else:
                word = lmtzr.lemmatize(i.strip(),get_simple_pos(pos[0][1]))
                final_text.append(word.lower())
    return " ".join(final_text)

In [16]:
review_data['text'] = review_data['text'].apply(lambda x: lemmatize_words(x))
review_data['text'][1]

'jake product exactly quite affordable realize double screen arrive even well expect add bonus screen carry small hint smell old grape candy use buy reminiscent sake stop put pop filter next nose smell record dif need pop filter work well expensive one even come please aroma mine buy product'

In [17]:
review_data.head()

,reviewerID,asin,reviewerName,helpful,overall,unixReviewTime,reviewTime,text
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]",5,1393545600,"02 28, 2014",good much write exactly suppose filter pop sou...
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",5,1363392000,"03 16, 2013",jake product exactly quite affordable realize ...
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",5,1377648000,"08 28, 2013",job well primary job device block breath other...
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",5,1392336000,"02 14, 2014",good windscreen money nice windscreen protects...
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",5,1392940800,"02 21, 2014",pop record vocal pop filter great look perform...


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_data = tfidf.fit_transform(review_data['text'])

In [20]:
X_data

<10261x15777 sparse matrix of type '<class 'numpy.float64'>'
	with 359658 stored elements in Compressed Sparse Row format>

In [19]:
y_data = review_data['overall']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state= 45)

In [22]:
from keras.utils import np_utils
y_train_updated = pd.get_dummies(y_train).values
y_test_updated = pd.get_dummies(y_test).values
num_classes = y_test_updated.shape[1]

Using TensorFlow backend.


In [132]:
num_classes

5

In [39]:
# Create the model
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras import Sequential

model = Sequential()


# Input - Layer
model.add(Dense(50, activation = "relu", input_shape=(15777, )))
# Hidden - Layers
model.add(Dropout(0.3, noise_shape=None, seed=None))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.3, noise_shape=None, seed=None))
model.add(Dense(50, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(50, activation = "relu"))
# Output- Layer
model.add(Dense(5, activation = "sigmoid"))
model.summary()

# model.add(Dense(units = 75 , activation = 'relu' , input_dim = X_train.shape[1]))
# model.add(Dense(units = 50 , activation = 'relu'))
# model.add(Dense(units = 25 , activation = 'relu'))
# model.add(Dense(units = 10 , activation = 'relu')) 
# model.add(Dense(units = 5, activation = 'sigmoid'))
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics=['accuracy'])


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 50)                788900    
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 50)                2550      
_________________________________________________________________
dropout_6 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 50)                2550      
_________________________________________________________________
dropout_7 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 50)               

In [152]:
X_train.shape[1:]

(15777,)

In [158]:
# model.add(Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())

In [ ]:
# model.add(Conv2D(64, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())

In [ ]:
# model.add(Conv2D(128, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())

In [ ]:
# model.add(Flatten())
# model.add(Dropout(0.2))

In [ ]:
# model.add(Dense(256, kernel_constraint=maxnorm(3)))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())

In [ ]:
# model.add(Dense(128, kernel_constraint=maxnorm(3)))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())

In [ ]:
# model.add(Dense(num_classes))
# model.add(Activation('softmax'))

In [40]:
epochs = 10
# optimizer = 'Adam'

In [ ]:
# model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [41]:
model.fit(X_train, y_train_updated, validation_data=(X_test, y_test_updated), epochs=epochs, batch_size=64)

Train on 7695 samples, validate on 2566 samples
Epoch 1/10
7695/7695 [==============================] - 6s 766us/step - loss: 0.4077 - accuracy: 0.8623 - val_loss: 0.3022 - val_accuracy: 0.8758
Epoch 2/10
7695/7695 [==============================] - 5s 630us/step - loss: 0.2897 - accuracy: 0.8707 - val_loss: 0.2756 - val_accuracy: 0.8883
Epoch 3/10
7695/7695 [==============================] - 5s 630us/step - loss: 0.2361 - accuracy: 0.9028 - val_loss: 0.2983 - val_accuracy: 0.8846
Epoch 4/10
7695/7695 [==============================] - 5s 593us/step - loss: 0.1959 - accuracy: 0.9146 - val_loss: 0.3364 - val_accuracy: 0.8775
Epoch 5/10
7695/7695 [==============================] - 5s 630us/step - loss: 0.1628 - accuracy: 0.9311 - val_loss: 0.4059 - val_accuracy: 0.8641
Epoch 6/10
7695/7695 [==============================] - 5s 617us/step - loss: 0.1353 - accuracy: 0.9447 - val_loss: 0.4618 - val_accuracy: 0.8521
Epoch 7/10
7695/7695 [==============================] - 5s 630us/step - loss

In [42]:
scores = model.evaluate(X_test, y_test_updated, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 84.75%


In [43]:
y_test_updated

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0]], dtype=uint8)

In [44]:
yhat_probs = model.predict(X_test)

In [107]:
# yhat_classes = model.predict_classes(X_test, verbose=0)

In [45]:
yhat_probs

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 5.9604645e-08,
        9.9999988e-01],
       [8.9406967e-08, 4.4703484e-07, 5.9306622e-06, 3.1488240e-03,
        9.9604249e-01],
       [3.9145350e-04, 2.1550059e-03, 2.4594605e-02, 9.7156632e-01,
        1.3204634e-02],
       ...,
       [3.1530857e-05, 1.4689565e-04, 7.7000260e-04, 3.9430887e-02,
        9.4800723e-01],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 2.0861626e-07,
        9.9999970e-01],
       [0.0000000e+00, 0.0000000e+00, 5.9604645e-08, 4.1974452e-04,
        9.9956876e-01]], dtype=float32)

In [46]:
idx = np.argmax(yhat_probs, axis=-1)
yhat_probs = np.zeros( yhat_probs.shape )
yhat_probs[ np.arange(yhat_probs.shape[0]), idx] = 1
yhat_probs

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [150]:
# # reduce to 1d array
# yhat_probs = yhat_probs[:, 0]
# # yhat_classes = yhat_classes[:, 0]
# yhat_probs

In [149]:
# y_test

In [47]:
target_names = ['class 1', 'class 2', 'class 3', 'class 4','class 5']
print(classification_report(y_test_updated, yhat_probs ,target_names = target_names))

              precision    recall  f1-score   support

     class 1       0.00      0.00      0.00        49
     class 2       0.00      0.00      0.00        60
     class 3       0.22      0.22      0.22       184
     class 4       0.29      0.42      0.34       504
     class 5       0.80      0.74      0.77      1769

   micro avg       0.61      0.61      0.61      2566
   macro avg       0.26      0.28      0.27      2566
weighted avg       0.62      0.61      0.61      2566
 samples avg       0.61      0.61      0.61      2566



In [128]:
# # print('Accuracy: %f' % accuracy)
# # precision tp / (tp + fp)
# precision = precision_score(y_test, yhat_classes,average = None)
# # print('Precision: %f' % precision)
# # recall: tp / (tp + fn)
# recall = recall_score(y_test, yhat_classes,average = None)
# # print('Recall: %f' % recall)
# # f1: 2 tp / (2 tp + fp + fn)
# f1 = f1_score(y_test, yhat_classes,average = None)
# # print('F1 score: %f' % f1)

## Model Selection

### LSTM

In [159]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

embedding_size=32
max_words=5000

model = Sequential()
model.add(Embedding(max_words, embedding_size, input_length=X_train.shape[1]))
model.add(LSTM(100))
model.add(Dense(5,activation='softmax'))

print(model.summary())

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 15777, 32)         160000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_47 (Dense)             (None, 5)                 505       
Total params: 213,705
Trainable params: 213,705
Non-trainable params: 0
_________________________________________________________________
None


In [160]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [24]:
# converting categorical variables in y_train to numerical variables
y_train_dummies = pd.get_dummies(y_train).values
print('Shape of Label tensor: ', y_train_dummies.shape)

Shape of Label tensor:  (7695, 5)


In [25]:
#train the model
model.fit(X_train, y_train_updated, epochs=5, batch_size=32)
#model.sav('MusicalInstrumentReviews.h5')

Epoch 1/5
 928/7695 [==>...........................] - ETA: 14:33:53 - loss: 1.1392 - accuracy: 0.6670

KeyboardInterrupt: 

In [ ]:
model.save('MusicalInstrumentReviews.h5')

In [ ]:
# converting categorical variables in y_train to numerical variables
y_test_dummies = pd.get_dummies(y_test).values
print('Shape of Label tensor: ', y_test_dummies.shape)

In [ ]:
# model evaluation
from keras.models import load_model

model = load_model('MusicalInstrumentReviews.h5')
scores = model.evaluate(X_test, y_test_dummies)

LSTM_accuracy = scores[1]*100

print('Test accuracy: ', scores[1]*100, '%')

## Naive Bayes

In [ ]:
# Instatiation, fitting and prediction

MNB = MultinomialNB()
MNB.fit(X_train, y_train)
predictions = MNB.predict(X_test)

In [ ]:
# Model evaluation

print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

MNB_f1 = round(f1_score(y_test, predictions, average='weighted'), 3)
MNB_accuracy = round((accuracy_score(y_test, predictions)*100),2)

print("Accuracy : " , MNB_accuracy , " %")
print("f1_score : " , MNB_f1)

## XGBoost

In [ ]:
# Instatiation, fitting and predictions

xgb_ = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softmax',
 nthread=4,
 scale_pos_weight=1,
 seed= 45)

xgb_.fit(X_train, y_train)
predictions = xgb_.predict(X_test)

In [ ]:
# Model evaluation
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

xgb_f1 = round(f1_score(y_test, predictions, average= 'weighted'), 3)
xgb_accuracy = round((accuracy_score(y_test, predictions) * 100), 2)

print("Accuracy : " , xgb_accuracy , " %")
print("f1_score : " , xgb_f1)